## 2일차 내용 이어서

In [1]:
### 예제 _ 프랜차이즈 전산실
# CITY, SUPPLIER, PART, SELLS
# CITY: PK, NAME
# SUPPLIER; PK, NAME, FK(CITY.PK)
# PARK: PK, NAME
# SELLES: FK(S.PK), FK(P.PK), PRICE

In [2]:
import sqlite3

In [3]:
con = sqlite3.connect('test1.db')
cur = con.cursor()

# 0306 시작

In [4]:
cur.execute('''
    select city.name, supplier.name from city
    left join supplier
    on city.pk = supplier.fk
    ''')
cur.fetchall()

[('성북구', '안암1호점'),
 ('성북구', '안암2호점'),
 ('성북구', '종암1호점'),
 ('중구', None),
 ('강북구', None),
 ('어쩌구', None),
 ('저쩌구', None)]

In [5]:
cur.execute('SELECT * FROM city')
cur.fetchall()

[(1, '성북구'), (2, '중구'), (3, '강북구'), (4, '어쩌구'), (5, '저쩌구')]

In [6]:
cur.execute('''
    select city.name, supplier.name from city
    right join supplier
    on city.pk = supplier.fk
    ''')
cur.fetchall()

OperationalError: RIGHT and FULL OUTER JOINs are not currently supported

In [7]:
cur.execute('''
    select city.name, supplier.name from city
    left join supplier
    on city.pk = supplier.fk
    ''')
cur.fetchall()

[('성북구', '안암1호점'),
 ('성북구', '안암2호점'),
 ('성북구', '종암1호점'),
 ('중구', None),
 ('강북구', None),
 ('어쩌구', None),
 ('저쩌구', None)]

In [8]:
cur.execute('''
    select city.name, supplier.name from city
    outer join supplier
    on city.pk = supplier.fk
    ''')
cur.fetchall()

OperationalError: RIGHT and FULL OUTER JOINs are not currently supported

In [21]:
con.close()

In [22]:
con = sqlite3.connect('playlist.db')
cur = con.cursor()

In [43]:
cur.executescript('''
    DROP TABLE IF EXISTS artist;
    DROP TABLE IF EXISTS album;
    DROP TABLE IF EXISTS genre;
    DROP TABLE IF EXISTS track;
    
    create table artist (
        pk integer primary key,
        name text default '무명'
    );
    
    create table album (
        pk integer primary key,
        name text default '무제',
        fk integer not null
    );
    
    create table genre (
        pk integer primary key,
        name text default '장르없음'
    );
    
    create table track (
        pk integer primary key,
        name text default 'Track',
        length integer default 0,
        rating integer default 0,
        count integer default 0,
        fk1 integer not null,
        fk2 integer not null
    );
''')

In [44]:
cur.execute('insert into artist values(?,?)', [None, '가수1'])

In [45]:
cur.execute('select * from artist')
cur.fetchall()

[(1, '가수1')]

In [46]:
cur.execute('insert into artist values(:pk, :name)',
           {'pk':None, 'name':'가수2'})

In [47]:
cur.execute('select * from artist')
cur.fetchall()

[(1, '가수1'), (2, '가수2')]

In [48]:
data = [['가수3'], ['가수4']]
cur.executemany('insert into artist(name) values(?)', data)

In [49]:
cur.execute('select * from artist')
cur.fetchall()

[(1, '가수1'), (2, '가수2'), (3, '가수3'), (4, '가수4')]

In [50]:
cur.lastrowid

4

In [51]:
cur.executescript('''
    insert into genre(pk, name) values(null, '장르1');
    insert into genre(name) values('장르2');
    insert into genre values(null, '장르3');
    insert into genre values(null, '장르4');
''')

In [52]:
cur.execute('select * from genre')
cur.fetchall()

[(1, '장르1'), (2, '장르2'), (3, '장르3'), (4, '장르4')]

In [53]:
cur.execute('select * from artist')
artist = cur.fetchall()

In [54]:
cur.execute('select * from artist where name=?', ['가수1'])
cur.fetchall()

[(1, '가수1')]

In [55]:
cur.execute('select * from artist where name like ?', ['%1'])
cur.fetchall()

[(1, '가수1')]

In [56]:
data = ['1', '2', '3', '4']

for val in data:
    cur.execute('select pk from artist where name like ?', ['%'+val])
    pk = cur.fetchall()
    if len(pk) > 0:
#         cur.execute('insert into album(pk,name, fk) values(?,?,?)')
#         cur.execute('insert into album values(null,?,?)')
#         cur.execute('insert into album(name, fk) values(?,?)')
        cur.execute('insert into album values(null,?,?)',
                   ['앨범'+val, pk[0][0]])

In [57]:
cur.execute('select * from album')
cur.fetchall()

[(1, '앨범1', 1), (2, '앨범2', 2), (3, '앨범3', 3), (4, '앨범4', 4)]

In [58]:
data = [('싱글1', '%1'), ('싱글2', '%2'), ('싱글3', '%3'), ('싱글4', '%4')]

cur.executemany('''
    insert into album(name, fk) values(?,(
        select pk from artist where name like ?
    ))
''', data)

In [59]:
cur.execute('select * from album')
cur.fetchall()

[(1, '앨범1', 1),
 (2, '앨범2', 2),
 (3, '앨범3', 3),
 (4, '앨범4', 4),
 (5, '싱글1', 1),
 (6, '싱글2', 2),
 (7, '싱글3', 3),
 (8, '싱글4', 4)]

In [61]:
cur.execute('select * from album')
FK1 = cur.fetchall()

cur.execute('select * from genre')
FK2 = cur.fetchall()

In [64]:
for row in FK1:
    if row[1] == '싱글1':
        fk1 = row[0]
        break

In [68]:
for row in FK2:
    if row[1] == '장르3':
        fk2 = row[0]
        break

In [70]:
cur.execute('''
    insert into track(pk, name, length, rating, count, fk1, fk2)
    values(null, ?, ?, ?, ?, ?, ?)
''', ['노래1', 270, 5, 100, fk1, fk2])
# map, filter 안 배웠으니 이렇게 static하게 해줄거다
# 원래는 map과 filter를 사용해야한다.
# 원래는 위의 두개의 for문이 한 줄이면 끝난다.

In [71]:
cur.execute('select * from track')
cur.fetchall()

[(1, '노래1', 270, 5, 100, 5, 3)]

In [72]:
cur.execute('''
    insert into track(name, fk1, fk2)
    values(?, ?, ?)
''', ['노래2', fk1, fk2])
# map, filter

In [73]:
cur.execute('select * from track')
cur.fetchall()

[(1, '노래1', 270, 5, 100, 5, 3), (2, '노래2', 0, 0, 0, 5, 3)]

In [75]:
cur.execute('insert into track(name, fk1, fk2) values(?,?,?)', 
            ['노래2', 1, 2])

In [76]:
cur.execute('select * from track')
cur.fetchall()

[(1, '노래1', 270, 5, 100, 5, 3),
 (2, '노래2', 0, 0, 0, 5, 3),
 (3, '노래2', 0, 0, 0, 1, 2)]

In [77]:
cur.execute('''
    insert into track(name, fk1, fk2) values(?,?,(
        select pk from genre where name like ?
    ))
''', ['노래3', 2, '%4'])

In [78]:
cur.execute('select * from track')
cur.fetchall()

[(1, '노래1', 270, 5, 100, 5, 3),
 (2, '노래2', 0, 0, 0, 5, 3),
 (3, '노래2', 0, 0, 0, 1, 2),
 (4, '노래3', 0, 0, 0, 2, 4)]

In [79]:
cur.execute('''
    insert into track(name, fk1, fk2) values(?,(
        select pk from album where name like ?
        ),(
        select pk from genre where name like ?
        ))
''', ['노래4', '%싱글%1%', '%장르%3%'])

In [85]:
data = [
    ['노래5', '%싱글%3%', '%장르%4%'],
    ['노래6', '%싱글%1%', '%장르%1%'],
    ['노래7', '%싱글%2%', '%장르%2%'],
    ['노래8', '%싱글%3%', '%장르%3%'],
    ['노래9', '%싱글%4%', '%장르%4%'],
    ['노래10', '%싱글%4%', '%장르%3%'],
    ['노래11', '%싱글%3%', '%장르%2%'],
    ['노래12', '%싱글%2%', '%장르%1%']
]
cur.executemany('''
    insert into track(name, fk1, fk2) values(?,(
        select pk from album where name like ?
    ),(
        select pk from genre where name like ?
    ))
''', data)

In [86]:
cur.execute('select count(*) from track')
cur.fetchall()

[(13,)]

In [89]:
cur.execute('select count(*) from track group by fk1')
cur.fetchall()

[(1,), (1,), (4,), (2,), (3,), (2,)]

In [91]:
cur.execute('select fk2, count(*) from track group by fk2')
cur.fetchall()

[(1, 2), (2, 3), (3, 5), (4, 3)]

In [92]:
cur.execute('''
    select T_B.pk, T_B.name, T_A.B from genre as T_B
    inner join
    (select fk2 as A, count(*) as B from track group by fk2) as T_A
    on T_A.A = T_B.pk
''')
cur.fetchall()

[(1, '장르1', 2), (2, '장르2', 3), (3, '장르3', 5), (4, '장르4', 3)]

In [93]:
cur.execute('''
    select T_B.pk, T_B.name, T_A.B from genre as T_B
    inner join
    (select fk2 as A, count(*) as B from track group by fk2) as T_A
    on T_A.A = T_B.pk
    where T_A.B > 2
    order by T_B.pk desc
    limit 0,2
''')
cur.fetchall()

[(4, '장르4', 3), (3, '장르3', 5)]

In [ ]:
가수, 가수의 앨범 수 <- 출력하시오

In [94]:
cur.execute('''
    select T_A.pk, T_A.name, T_B.CNT from artist as T_A
    inner join
    (select fk, count(*) as CNT from album group by fk) as T_B
    on T_B.fk = T_A.pk
''')
cur.fetchall()

[(1, '가수1', 2), (2, '가수2', 2), (3, '가수3', 2), (4, '가수4', 2)]

In [96]:
cur.execute('''
    select track.pk, track.name, genre.name from track
    inner join genre on genre.pk = track.fk2
    order by genre.name asc
''')
cur.fetchall()

[(7, '노래6', '장르1'),
 (13, '노래12', '장르1'),
 (3, '노래2', '장르2'),
 (8, '노래7', '장르2'),
 (12, '노래11', '장르2'),
 (1, '노래1', '장르3'),
 (2, '노래2', '장르3'),
 (5, '노래4', '장르3'),
 (9, '노래8', '장르3'),
 (11, '노래10', '장르3'),
 (4, '노래3', '장르4'),
 (6, '노래5', '장르4'),
 (10, '노래9', '장르4')]

In [100]:
cur.execute('''
    select track.pk, track.name, album.name, genre.name from track
    inner join genre on genre.pk = track.fk2
    inner join album on album.pk = track.fk1
    order by album.name, genre.name
''')
cur.fetchall()

[(7, '노래6', '싱글1', '장르1'),
 (1, '노래1', '싱글1', '장르3'),
 (2, '노래2', '싱글1', '장르3'),
 (5, '노래4', '싱글1', '장르3'),
 (13, '노래12', '싱글2', '장르1'),
 (8, '노래7', '싱글2', '장르2'),
 (12, '노래11', '싱글3', '장르2'),
 (9, '노래8', '싱글3', '장르3'),
 (6, '노래5', '싱글3', '장르4'),
 (11, '노래10', '싱글4', '장르3'),
 (10, '노래9', '싱글4', '장르4'),
 (3, '노래2', '앨범1', '장르2'),
 (4, '노래3', '앨범2', '장르4')]

In [102]:
cur.execute('''
    select track.pk, artist.name, track.name, album.name, genre.name
    from track
    inner join genre on genre.pk = track.fk2
    inner join album on album.pk = track.fk1
    inner join artist on artist.pk = album.fk
    where artist.name like '%1'
    order by artist.name, album.name, genre.name
''')
cur.fetchall()

[(7, '가수1', '노래6', '싱글1', '장르1'),
 (1, '가수1', '노래1', '싱글1', '장르3'),
 (2, '가수1', '노래2', '싱글1', '장르3'),
 (5, '가수1', '노래4', '싱글1', '장르3'),
 (3, '가수1', '노래2', '앨범1', '장르2')]

In [104]:
# 위 코드와 같은 결과
cur.execute('''
    select track.pk, artist.name, track.name, album.name, genre.name
    from track
    inner join genre on genre.pk = track.fk2
    inner join album on album.pk = track.fk1
    inner join artist on artist.pk = album.fk AND artist.name like '%1'
    order by artist.name, album.name, genre.name
''')
cur.fetchall()

[(7, '가수1', '노래6', '싱글1', '장르1'),
 (1, '가수1', '노래1', '싱글1', '장르3'),
 (2, '가수1', '노래2', '싱글1', '장르3'),
 (5, '가수1', '노래4', '싱글1', '장르3'),
 (3, '가수1', '노래2', '앨범1', '장르2')]

In [105]:
# 위 코드와 같은 결과
cur.execute('''
    select track.pk, artist.name, track.name, album.name, genre.name
    from track
    inner join genre on genre.pk = track.fk2
    inner join album on album.pk = track.fk1
    inner join artist on artist.pk = album.fk
    where track.count < 10
    order by artist.name, album.name, genre.name
''')
cur.fetchall()

[(7, '가수1', '노래6', '싱글1', '장르1'),
 (2, '가수1', '노래2', '싱글1', '장르3'),
 (5, '가수1', '노래4', '싱글1', '장르3'),
 (3, '가수1', '노래2', '앨범1', '장르2'),
 (13, '가수2', '노래12', '싱글2', '장르1'),
 (8, '가수2', '노래7', '싱글2', '장르2'),
 (4, '가수2', '노래3', '앨범2', '장르4'),
 (12, '가수3', '노래11', '싱글3', '장르2'),
 (9, '가수3', '노래8', '싱글3', '장르3'),
 (6, '가수3', '노래5', '싱글3', '장르4'),
 (11, '가수4', '노래10', '싱글4', '장르3'),
 (10, '가수4', '노래9', '싱글4', '장르4')]

# ORM

데이터를 오브젝트로 매핑해준다.\
쿼리로 바꿔서 뒷단에서 왔다갔다해준다...\
웹에서 로그인했을 때 불러오는 정보들 관리할 때 많이 쓴다\
아마 웹을 하게 되면 이것을 쓸것이다 당신들이\
데이터베이스와 오브젝트를 연결하는 것이 ORM이다

1. 새 게시물 생성
2. 생성 시, 사용자 태그(0~N)
    3. 사용자 태그가 해시태그 풀에 잇는 지 확인
    4. 새 게시물 - 해시태그 관계 만들어
    5. 해시태그 풀에 있는 빈도 정보 +1

In [ ]:
posting - pk, title, content, date

hashtag - pk, name, count

posting-hashtag : posting.pk, hashtag.pk

In [106]:
con = sqlite3.connect('sns.db')
cur = con.cursor()

In [107]:
cur.execute('select current_timestamp')
cur.fetchall()

[('2023-03-06 02:50:47',)]

In [110]:
cur.executescript('''
    --drop table if exists posting;
    --drop table if exists hashtag;
    --drop table if exists poshas;
    create table posting (
        pk integer primary key,
        title text,
        content text,
        regdate date default current_timestamp
    );
    
    create table hashtag (
        pk integer primary key,
        name text,
        count integer default 0
    );
    
    create table poshas (
        fk1 integer not null,
        fk2 integer not null
    );
''')

In [111]:
# 1. 새 게시물 생성
# def addPosting(title, content, *hashtag)
cur.execute('insert into posting(title, content) values(?,?)',
            ['제목1', '내용1'])
pid = cur.lastrowid

In [112]:
pid

1

In [119]:
hashtag = ['태그1', '태그2']
tagids = list()

# 2. 생성 시, 사용자 태그(0~N)
#     3. 사용자 태그가 해시태그 풀에 잇는 지 확인

for tag in hashtag:
    cur.execute('select pk from hashtag where name=?', [tag])
#     print(cur.fetchone()) # 없으면 None
    tid = cur.fetchone()
    if tid is not None:
        tagids.append(tid[0])
tagids

[1, 2]

In [120]:
#     4. 새 게시물 - 해시태그 관계 만들어
#     5. 해시태그 풀에 있는 빈도 정보 +1

for tid in tagids:
    cur.execute('''
        insert into poshas values(?,?)
    ''', [pid, tid])
        
    cur.execute('''
        update hashtag
        set count = count + 1
        where pk=?
    ''', [tid])

In [118]:
# 태그풀 생성
tags = [['태그1'], ['태그2'], ['태그3']]
cur.executemany('insert into hashtag(name) values(?)', tags)

In [121]:
# commit을 해야 db에 보인다
con.commit()

In [123]:
# 잘 들어갔는지 확인
cur.execute('''
    select posting.title, posting.content, posting.regdate, hashtag.name
    from poshas
    inner join posting on posting.pk = fk1
    inner join hashtag on hashtag.pk = fk2
''')
cur.fetchall()

[('제목1', '내용1', '2023-03-06 02:57:27', '태그1'),
 ('제목1', '내용1', '2023-03-06 02:57:27', '태그2')]

In [127]:
# 만약에 내가 태그 정보가 필요하다고 한다면
cur.execute('''
    select hashtag.name
    from hashtag
    inner join posting on posting.pk = fk1
    inner join hashtag on hashtag.pk = fk2
    where fk1 = ?
''', pid)
cur.fetchall()

OperationalError: ambiguous column name: hashtag.name

In [ ]:
# 수정
# 제목1, 내용1, 태그1, 태그2
# -> 제목1-1, 내용1, 태그1, 태그3
cur.execute('update posting set title=?, content=?',
           ['제목1-1', '내용1'])

In [ ]:
cur.rowcount

In [ ]:
hashtag = ['태그1', '태그3']
oldtag = list()
tagids = list()


cur.execute('select fk2 from poshas where fk1=?', [pid])
for row in cur.fetchall():
    oldtag.append(row[0])
    
    
for tag in hashtag:
    cur.execute('select pk from hashtag where name=?', [tag])
#     print(cur.fetchone()) # 없으면 None
    tid = cur.fetchone()
    if tid is not None:
        tagids.append(tid[0])
oldtag, tagids

In [ ]:
for tid in oldtag:
    if tid not in tagids: # 없으면 삭제하고 카운트 -1 해준다
        cur.execute('delete from poshas where fk1=? and fk2=?',
                   [pid, tid])
        cur.execute('update hashtag set count = count - 1 where pk=?',
                   [tid])

In [ ]:
for tid in oldtag:
    if tid not in tagids:
        cur.execute('insert into poshas values(?,?)',
                   [pid, tid])
        
        cur.execute('update hashtag set count = count + 1 where pk=?',
                   [tid])

In [ ]:
cur.execute('select * from poshas')
cur.fetchall()

In [ ]:
cur.execute('select * from hashtag')
cur.fetchall()

In [ ]:
con.close()